In [1]:
%pip install -r requirements.txt > NUL 2>&1

Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
import os

from dotenv import load_dotenv
warnings.filterwarnings("ignore")


In [3]:
load_dotenv()
api_key = os.getenv("API_KEY")

## 1. Load Model

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Ateeqq/product-description-generator", use_auth_token=api_key)
model = AutoModelForSeq2SeqLM.from_pretrained("Ateeqq/product-description-generator", use_auth_token=api_key)
#model = 
#model = 
print('Model loaded successfully :)')

Model loaded successfully :)


## 2. Create Prompt

In [5]:
def generate_custom_prompt(title, primary_color, secondary_color, features):
    # Create the prompt by incorporating user input
    prompt = (
        f"Description for {title}: "
        f"A product that features {primary_color} and {secondary_color} colors. "
        f"Key unique features include {features[0]}, {features[1]}, and {features[2]}. "
        f"Generate a compelling and informative product description highlighting these attributes."
    )
    return prompt

## 3. Generate Description

In [6]:
def generate_product_description(title, primary_color, secondary_color, features):
    # Create the custom prompt based on user input
    custom_prompt = generate_custom_prompt(title, primary_color, secondary_color, features)
    
    # Encode the custom prompt
    input_ids = tokenizer(custom_prompt, return_tensors="pt", padding="longest", truncation=True, max_length=128).input_ids.to(model.device)
    
    # Generate product descriptions based on the custom prompt
    outputs = model.generate(
        input_ids,
        num_beams=5,                # Beam search for diversity
        num_beam_groups=5,          # Split beams into groups for more diverse generations
        num_return_sequences=5,      # Return multiple descriptions
        repetition_penalty=10.0,     # Penalize repetitive phrases
        diversity_penalty=3.0,       # Encourage diverse outputs
        no_repeat_ngram_size=2,      # Prevent repeated phrases
        temperature=0.7,             # Control randomness of the generation
        max_length=128               # Maximum length of the generated description
    )
    
    # Decode the generated descriptions
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [7]:
# Example usage
title = "18-Piece Kitchen Dinnerware Set"
primary_color = "white"
secondary_color = "silver"
features = ["microwave-safe", "scratch-resistant", "stackable design"]

# Generate descriptions based on the custom prompt
descriptions = generate_product_description(title, primary_color, secondary_color, features)

In [8]:
# Print the generated descriptions
for idx, desc in enumerate(descriptions):
    print(f"Description {idx + 1}:")
    print(desc)
    print()

Description 1:
Description for 18-Piece Kitchen Dinnerware Set: A product that features white and silver colors. Key unique features include microwave-safe, scratch-resistant, stackable design. Generate a compelling and informative product description highlighting these attributes. The name is printed on the back of each item to help you identify your items in order to make sure they are displayed correctly. This listing includes one set of dinnerware with an attractive black color; this tableware has been designed by our team from its beginnings as if it were made out of high quality materials. It's also very easy to clean because there

Description 2:
Description For 18 Piece Kitchen Dinnerware Set: A product that features white and silver colors. Key unique features include microwave-safe, scratch resistant, stackable design. Generate a compelling and informative product description featuring these attributes. This is an excellent gift for your family or friends who love cooking wit

In [9]:
# Select and print the best description without saving
def select_description(descriptions):
    # Ask the user to select the preferred description
    selection = int(input("Select the description you like (1-5): ")) - 1
    
    if 0 <= selection < len(descriptions):
        # Print the selected description instead of saving
        selected_description = descriptions[selection]
        print(f"\nDescription {selection + 1} selected: {selected_description}")
    else:
        print("Invalid selection. Please try again.")

# Call the function to select the description
select_description(descriptions)

## 4. SEO Generation